In [ ]:
from airflow.contrib.hooks.sftp_hook import SFTPHook
import io
import pandas as pd 
import datetime as dt
from #directory.api import api


def import_client_data():
    sftp_conn = SFTPHook('database').get_conn()
    base_path =  'path'
    all_file_names = sftp_conn.listdir(base_path)
    
    if not all_file_names:
        return None
    
    file_name = all_file_names[0]
    print(file_name)
    file_content = io.BytesIO()
    sftp_conn.getfo(base_path + file_name, file_content)
    file_content.seek(0)


    # layout with position of the values to search by row
    layout = {
        'nome': (178, 218),
        'sexo': (218, 219),
        'data_nascimento': (219, 227),
        'cpf_cnpj': (164, 178),
        'email': (759, 799),
        'logradouro': (294, 334),
        'numero': (334, 339), 
        'complemento': (339, 354),
        'bairro': (354, 374),
        'cidade': (382, 402),
        'uf': (402, 404),
        'cep': (374, 382),
        'ddd': (725, 730),
        'telefone': (404, 424),
        'celular': (730, 750),
    }

    df = pd.read_fwf(file_content, skiprows=1, header = None, names=layout.keys(), colspecs=list(layout.values()), encoding='latin1', dtype=str)

    id_origem = #insert id_origem
    id_status = 1
    id_resultado = 1

    df['cpf_cnpj'] = df['cpf_cnpj'].apply(lambda x: x[:3] if x.startswith('000') else None)
    df['sexo'] = df['sexo'].apply(lambda x: x.strip() if x != '0' else None)
    df['data_nascimento'] = df['data_nascimento'].apply(lambda x: x.strip() if x[:2] != '00' else None)
    df['data_nascimento'] = df['data_nascimento'].apply(lambda x: dt.datetime.strptime(x, "%d%m%Y").strftime('%d/%m/%Y') if x else None)

    df.loc[~df['email'].str.contains('@', na=False), 'email'] = None

    df['cep'] = df['cep'].apply(lambda x: x.strip() if x[:3] != '000' else None)

    for index, row in df.iterrows():        
        data = {
            'id_origem': id_origem,
            'nome': row['nome'],
            'cpf_cnpj': row['cpf_cnpj'],
            'sexo': row['sexo'],
            'data_nascimento': row['data_nascimento'],
            'emails': [],

            'enderecos': [
                {
                'logradouro': row['logradouro'],
                'numero': row['numero'],
                'complemento': row['complemento'],
                'bairro': row['bairro'],
                'cidade': row['cidade'],
                'uf': row['uf'],
                'cep': row['cep'],
                'id_status': id_status
                }
            ],
            'ligacoes': [
                {'ddd': row['ddd'], 'numero': row['telefone'], 'id_resultado': id_resultado},
                {'ddd': row['ddd'], 'numero': row['celular'], 'id_resultado': id_resultado}
            ]

        }
        
        if row['email']: 
            data['emails'].append({'email': row['email'], 'id_status': id_status})

    
        try:
            res = #api.function_get_client(data)
            print(res)
        except Exception as e:
            print(e)
            

    sftp_conn.remove(base_path+file_name)

